<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module06/prop01_PP_ETP_resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine-python/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

### **Ejercicio Propuesto: Estimar la precipitacion y evapotranspiracion areal climatica mensual para todos los distritos del departamento de Amazonas, Peru* (Opcional)**


[Respuesta aqui](https://gist.github.com/csaybar/d10c67ada2506842247c9419320d5051)

El resultado final debe ser un **SHAPEFILE (o GeoJSON)** que presente una tabla que indique:

1) El nombre del departamento, distrito y provincia.

2) **12** campos referidos a precipitacion climatica mensual.

3) **12** campos referidos a evapotranspiracion climatica mensual.

**La tarea se considerara como concluida luego de enviar un print de pantalla con la informacion alfanumerica del shapefile generado al correo indicado en la plataforma: tareas@mastergis.com**

La imagen inferior es un ejemplo de lo que se debe adjuntar en el mensaje
<center>
<img src='https://user-images.githubusercontent.com/16768318/73798741-b6330c80-47ab-11ea-938a-2faee3eb1c89.png' width=100%>

</center>


In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_TU_TOKEN_AQUI"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1. Cargar datos vectoriales**

In [0]:
amazonas = ee.FeatureCollection('users/csaybardemo/amazonas')
amazonas_img = amazonas.draw(color = "000000", strokeWidth = 3, pointRadius = 3)
center = [-78.05,-5.07]
mapdisplay(center,{'amazonas':amazonas_img.getMapId()})

### **2. Cargar datos raster (Imagenes)**

In [0]:
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                 .filterDate("1981-01-01", "2009-12-31")

### **3. Extraer los valores**
Utiliza reduce y map para solucionar este ejercicio!

In [0]:
import re

# Extraer valores en GEE por geometrias (puntos, lineas y poligonos) pareciera
# una tarea facil, sin embargo, es necesario un conocimiento claro de como 
# funciona la API.
def extract_values(x, y, fun = ee.Reducer.mean(), scale=1000,ID=None):
  
  # Hacemos un backup del id de la geometria
  if ID is None:
    y = ee.FeatureCollection(y).map(
        lambda x: x.set("ee_ID", x.get("system:index")))
  else:
    y = ee.FeatureCollection(y).map(
        lambda x: x.set("ee_ID", x.get(ID)))
  
  #Elimina la palabra "Reducer." para quedarte solo con el tipo de reductor
  fun_name = re.sub("Reducer.", "", fun.getInfo()["type"])

  #Reducimos por region a cada imagen de nuestro ImageCollection
  triplets = ee.ImageCollection(x).map(lambda image:image.reduceRegions(
      collection = y,
      reducer = fun,
      scale = scale
    ).map(lambda f: f.set("imageId", image.id()))).flatten()
  table = table_format(table = triplets,
                       rowId = "ee_ID", 
                       colId = "imageId",
                       val = fun_name)#.map(lambda f:f.setGeometry(None))
  return table



def table_format(table, rowId, colId,val):
    """Format a table of triplets into a 2D table of rowId x colId.
    Args:
        table: EE Image or ImageCollection
        rowID: ID unico de las geometrias.
        colID: ID unico de las imagenes.
        fun_name: Nombre de la funcion aplicada
    Returns:
        A EE table
    """
    # Obtenga la lista de todas las ID de geometrias unicas.
    rows = table.distinct(rowId)
    
    # Hasta aqui "table" esta agrupado por espacio es decir en el primer grupo
    # estan todas las geometrias para el mes de enero, en el segundo grupo para
    # febrero etc.
    # Con ee.Join.SaveAll cambiaremos este orden agrupando temporalmente es 
    # decir ahora el primer grupo tendra todos los meses de la primera 
    # geometria, el segun grupo todos los meses de la segunda geometria, etc.    
    joined = ee.Join.saveAll('matches').apply(
        primary =  rows,
        secondary = table,
        condition = ee.Filter.equals(
          leftField = rowId,
          rightField = rowId
        ))
    row = joined.first()
    def get_f(row):
        # Obtenga la lista de todas las grupos (geometrias) con un ID de fila unico y
        # Mapear una funcion sobre la lista de filas para devolver una lista de
        # ID de columna y valor.
        values = ee.List(row.get('matches')).map(lambda feature: [
          ee.Feature(feature).get(colId),
          ee.Feature(feature).get(val)]) # val es el resultado de la reduccion
        # Devuelve la fila con su propiedad ID y propiedades para
        # todas las ID de columnas coincidentes que almacenan la salida del reductor.
        # El constructor del diccionario está usando una lista de pares clave, valor.
        return row.select([rowId]).set(ee.Dictionary(values.flatten()))
    return joined.map(get_f)

### **4. Descargar los resultados (De Google Earth Engine a Google Drive)**
**ee.batch.Export.table.toDrive():** Guarda FeatureCollection como shapefile en Google Drive.

**ee.batch.Export.image.toDrive():** Guarda Images como GeoTIFF en Google Drive.

In [0]:
# Donde
# collection: es el FeatureCollection con la informacion climatica
# description: es el nombre que tendra el archivo SHP en Google Drive.
# folder: es la carpeta que se creara en Google Drive.
# fileFormat: es el formato del archivo de salida ('GeoJSON','KML','KMZ','SHP','CSV','TFRecord')
amazon_pp_result  = extract_values(terraclimate.select('pr'), amazonas, fun = ee.Reducer.mean(), scale=1000)
amazon_aet_result  = extract_values(terraclimate.select('aet'), amazonas, fun = ee.Reducer.mean(), scale=1000)

task = ee.batch.Export.table.toDrive(
    collection=amazon_pp_result,
    description='amazon_pp_climatic',
    folder='TareaMASTERGIS',
    fileFormat='GeoJSON')
task.start()

In [0]:
#@title Mensage Final del curso
%%html
<marquee style='width: 30%; color: blue;'><b>MUCHAS GRACIAS ESPERO TE HAYAS DIVERTIDO TOMANDO ESTE CURSO :3 ... HASTA UNA PROXIMA OPORTUNIDAD</b></marquee>